## VASP stresses

In [7]:
import numpy as np
from pymatgen.io.vasp.outputs import Outcar

In [8]:
def stresses_from_outcar( filename='OUTCAR' ):
    """Finds and returns stress tensors from the OUTCAR file.
      
    Args:
        filename (:obj:'str', optional): the name of the ``OUTCAR`` file to be read. Default is `OUTCAR`.
    Returns:
        (np.array): The stresses as found in the ``OUTCAR`` file in kB, as a 1D numpy array.
    """
    outcar = Outcar(filename)
    stresses = outcar.read_table_pattern(
        header_pattern=r"\s+Fock(\s+[+-]?\d+\.\d+)*\n\s+-+\n\s+Total(\s+[+-]?\d+\.\d+)*",
        row_pattern=r"\s+\D+\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)",
        footer_pattern=r"\s+external.+",
        postprocess=lambda x: float(x),
        last_one_only=False
    )
    return np.array(stresses).flatten()

In [31]:
#Units of kBar if wanting units of eV/(cell volume) remove `\n\s+Total(\s+[+-]?\d+\.\d+)*` from header_pattern and change 'external' to 'in kB' in footer_pattern
stresses = stresses_from_outcar( 'outcars/thermos/OUTCAR4' )

In [16]:
stresses #XX YY ZZ XY YZ ZX 

array([ 9.257303e+01,  1.604443e+01,  5.175611e+01,  5.479450e+00,
       -3.362590e+00, -1.631000e-02])

In [32]:
stresses

array([112.1004 ,  12.27436,  58.05216,  -1.90618,  -6.71627,  -5.74808])

## LAMMPS stresses
Units = pressure (bars) /1000 to convert to kB
format = XX YY ZZ XY XZ YZ 

In [ ]:
#Units of Bar *1000 = kBar
lammps_stresses = lammps.thermo.computes['thermo_press'].vector / 1000 #xx, yy, zz, xy, xz, yz
lammps_stresses[5], lammps_stresses[4] = lammps_stresses[4], lammps_stresses[5] #xx, yy, zz, xy, yz, xz

In [ ]:
lammps_stresses

# Testing

In [1]:
import os
from fitting import FitModel
from scipy import optimize
import numpy as np
import json

In [2]:
def get_forces(fit_data, values, args):
    fit_data.init_potential(values, args)
    ip_forces = fit_data.get_forces()
    dft_forces = fit_data.expected_forces()
    return dft_forces, ip_forces

In [3]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0, #+0.960,
                           'shell': 0.0}} #-2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['cs_springs'] = {'O-O' : [20.0, 0.0]}

distribution = {}
distribution['Li-O'] = {'bpp' : [663.111, 0.119, 0.0],
                        'sd' : [80, 0.01, 0.01]}
distribution['Ni-O'] = {'bpp' : [1393.540, 0.218, 0.000],
                        'sd'  : [80, 0.01, 0.01]}
distribution['O-O'] = {'bpp' : [25804.807, 0.284, 0.0],
                       'sd'  : [200, 0.01, 5]}

In [4]:
include_labels = ['dq_O', 'q_scaling', 'O-O spring', 'Li_O_a',     'Li_O_rho', 'Ni_O_a',      'Ni_O_rho', 'O_O_a',        'O_O_rho']
include_values = [3.0388, 0.5839, 11.3029, 21136.1895, 0.1648, 18779.3350, 0.1748, 19279.4957, 0.2012]

In [5]:
fit_data = FitModel.collect_info(params, distribution, supercell=[2,2,2])
# s = optimize.differential_evolution(fit_data.chi_squared_error, bounds=bounds_list, popsize=25,
#                                     args=([include_labels]), maxiter=2000,
#                                     disp=True, init='latinhypercube', workers=-1)
# dft_forces, ip_forces = get_forces(fit_data, s.x, include_labels)

Found elements: ['Li', 'Ni', 'O']


In [6]:
fit_data.chi_squared_error_force_and_stress(values=include_values, args=include_labels)

[ 9.257303e+01  1.604443e+01  5.175611e+01  5.479450e+00 -3.362590e+00
 -1.631000e-02]


4144.317236780679

In [ ]:
for structure in fit_data.lammps_data:
    print(structure.expected_stress_tensors)